In [3]:
import pandas as pd
import numpy as np

In [4]:
# loading both the csv files
credit = pd.read_csv('C:/Users/adity/.vscode/.vscode/.vscode/vs_projects/mypythonprjt/jupyter notebooks_py/ML_prj/recommender_system/tmdb_5000_credits.csv')
movies = pd.read_csv('C:/Users/adity/.vscode/.vscode/.vscode/vs_projects/mypythonprjt/jupyter notebooks_py/ML_prj/recommender_system/tmdb_5000_movies.csv')


## merging and selecting useful features for my project
movies = movies.merge(how = 'inner' , left_on='title' , right_on= 'title' , right= credit)
removing_col = ['budget' , 'homepage' , 'original_language' , 'original_title' , 'popularity' , 'production_companies' , 'production_countries' , 'revenue' , 'runtime' , 'spoken_languages' , 'tagline' , 'vote_average' , 'release_date' , 'status' , 'vote_count']
movies = movies.drop(removing_col , axis = 1)

In [5]:
movies.dropna(inplace= True)
movies.sample(5)

,genres,id,keywords,overview,title,movie_id,cast,crew
1297,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 28, ""nam...",9531,"[{""id"": 83, ""name"": ""saving the world""}, {""id""...","Aiming to defeat the Man of Steel, wealthy exe...",Superman III,9531,"[{""cast_id"": 1, ""character"": ""Superman / Clark...","[{""credit_id"": ""5539a98fc3a368784500791d"", ""de..."
1353,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 878, ""...",782,"[{""id"": 856, ""name"": ""paraplegic""}, {""id"": 125...",Science fiction drama about a future society i...,Gattaca,782,"[{""cast_id"": 7, ""character"": ""Vincent Freeman""...","[{""credit_id"": ""52fe4275c3a36847f8020563"", ""de..."
2061,"[{""id"": 10749, ""name"": ""Romance""}, {""id"": 18, ...",21494,"[{""id"": 10965, ""name"": ""playwright""}]","Molière, a down-and-out actor-cum-playwright u...",Moliere,21494,"[{""cast_id"": 1, ""character"": ""Moliere"", ""credi...","[{""credit_id"": ""5900754bc3a3680fcc00a327"", ""de..."
3050,"[{""id"": 18, ""name"": ""Drama""}]",87093,"[{""id"": 1157, ""name"": ""wife husband relationsh...","The story of the awakening of the painter, Mar...",Big Eyes,87093,"[{""cast_id"": 8, ""character"": ""Margaret Keane"",...","[{""credit_id"": ""52fe49b59251416c910b622d"", ""de..."
1511,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 18, ""...",157841,"[{""id"": 13008, ""name"": ""train""}, {""id"": 214527...",A 12-year-old cartographer secretly leaves his...,The Young and Prodigious T.S. Spivet,157841,"[{""cast_id"": 16, ""character"": ""T.S. Spivet"", ""...","[{""credit_id"": ""52fe4bd49251416c910e6fc7"", ""de..."


In [6]:
print(movies.shape)
credit.shape

(4806, 8)


(4803, 4)

## extracting the tags from the important columns

In [7]:
def mine(obj):
    '''this function is to mine the necessary stuff from the list of dictionaries in some important columns of the dataset'''
    obj = eval(obj)
    core = []
    for i in range(len(obj)):
        core.append(obj[i]['name'])
        
    return core

def mine_2(obj):
    '''for cast column'''
    obj = eval(obj)
    core_2 = []
    try:
        for i in range(3):
            core_2.append(obj[i]['name'])
            
    except:
        try:
            for i in range(2):
                core_2.append(obj[i]['name'])
        except:
            try:
                for i in range(1):
                    core_2.append(obj[i]['name'])
            except:
                pass
        
    return core_2

def mine_3(obj):
    '''for crew column as i want to get the name of only with job vlaue director'''
    obj = eval(obj)
    core_3 = []
    a = 0
    try:
        while True:
            if obj[a]['job'] == 'Director':
                core_3.append(obj[a]['name'])
            a +=1
    except:
        pass
    
    return core_3

movies['genres'] = movies['genres'].apply(mine)
movies['keywords'] = movies['keywords'].apply(mine)
movies['cast'] = movies['cast'].apply(mine_2)
movies['crew'] = movies['crew'].apply(mine_3)
movies['overview'] = movies['overview'].apply(lambda x : x.split())
movies['genres'] = movies['genres'].apply(lambda x: [txt.replace(" ",'') for txt in x])
movies['cast'] = movies['cast'].apply(lambda x: [txt.replace(" ",'') for txt in x])
movies['crew'] = movies['crew'].apply(lambda x: [txt.replace(" ",'') for txt in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [txt.replace(" ",'') for txt in x])
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']


## getting and working with the new dataframe 'df'

In [8]:
new_df = movies.loc[:,['movie_id' , 'title' , 'tags']]
new_df['tags'] = new_df['tags'].apply(lambda x: (' '.join(x)).lower())

In [9]:
new_df.head(5)

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


## feature extraction

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize , sent_tokenize
from nltk.stem import PorterStemmer

In [11]:
ps = PorterStemmer()
def stemming(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
        
    return ' '.join(y)

new_df['tags'] = new_df['tags'].apply(stemming)

cv = CountVectorizer(stop_words= 'english' , max_features= 4000)
vectors = cv.fit_transform(new_df['tags']).toarray()

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
cos_theta = cosine_similarity(vectors)

In [18]:
def recommend(movie):
    index = (new_df[new_df['title'] == movie].index)
    type(index)
    similar_5_index = list(np.argsort(cos_theta[index]))[0][::-1][1:6]
    rec_movies = [new_df['title'][ind] for ind in similar_5_index]
    return rec_movies

recommend('Terminator 3')

IndexError: list index out of range